# Ground state solvers

In [30]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

es_problem = driver.run()

In [31]:
es_problem.reference_energy

-1.116998996754004

In [32]:
from qiskit.algorithms.minimum_eigensolvers import (
    VQE, AdaptVQE, NumPyMinimumEigensolver)
from qiskit.algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

import numpy as np

mapper = JordanWignerMapper()

In [33]:
solver = GroundStateEigensolver(mapper, NumPyMinimumEigensolver())
result = solver.solve(es_problem)
print(result)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030202
  - computed part:      -1.857275030202
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035753
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388948701555]
    - computed part:      [0.0  0.0  1.388948701555]
  > Dipole moment (a.u.): [0.0  0.0  -0.000000001555]  Total: 0.000000001555
                 (debye): [0.0  0.0  -0.000000003953]  Total: 0.000000003953
 


In [34]:

# mapper = QubitConverter(JordanWignerMapper())
mapper = JordanWignerMapper()

ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    ),
)

vqe_solver = VQE(Estimator(), ansatz, SLSQP())
vqe_solver.initial_point = np.zeros(ansatz.num_parameters)

adapt_vqe = AdaptVQE(vqe_solver)


In [35]:


vqe_calc = GroundStateEigensolver(mapper, vqe_solver)
vqe_res = vqe_calc.solve(es_problem)

adap_calc = GroundStateEigensolver(mapper, adapt_vqe)
adap_res = adap_calc.solve(es_problem)

print(f"VQE total ground state energy = {vqe_res.total_energies[0]:.4f}")
print(f"ADAPT-VQE total ground state energy = {adap_res.total_energies[0]:.4f}")

VQE total ground state energy = -1.1373
ADAPT-VQE total ground state energy = -1.1373


In [36]:
vqe_res.hartree_fock_energy

-1.116998996754004

In [37]:
es_problem.reference_energy

-1.116998996754004

In [38]:
import pandas as pd

dict = {'Name' : ['Martha', 'Tim', 'Rob', 'Georgia'],
        'Maths' : [87, 91, 97, 95],
        'Science' : [83, 99, 84, 76]}
df = pd.DataFrame(dict)
df

,Name,Maths,Science
0,Martha,87,83
1,Tim,91,99
2,Rob,97,84
3,Georgia,95,76


In [41]:
dict = {'Method' : ['Hartree-Fock', 'Exact', 'VQE'],
        'Ground state energy' : [es_problem.reference_energy, adap_res.total_energies[0], vqe_res.total_energies[0]],
        'Delta' : [83, 99, 8]}
df = pd.DataFrame(dict)
df

,Method,Ground state energy,Delta
0,Hartree-Fock,-1.116999,83
1,Exact,-1.137306,99
2,VQE,-1.137306,8


In [ ]:
from pyscf import gto, scf, mcscf
from pyscf.mcscf import avas #AVAS method to construct mcscf active space

from qiskit_nature_pyscf import QiskitSolver

In [ ]:
#  Initialisation de la structure moléculaire
H2_mol = gto.M(atom="H 0 0 0; H 0 0 .735", basis="sto-3g")

# Calculs HF 
H2_h_f = scf.RHF(H2_mol).run()

# Calculs post-HF
norb, nelec, mo =avas.avas(H2_h_f,['H 1s','H 1s'])

H2_cas = mcscf.CASCI(H2_h_f, norb, nelec)

# Intégration de l'algorithme quantique
# mapper = JordanWignerMapper()
# optimizer = SPSA()

H2_cas.fcisolver = QiskitSolver(adap_calc)

H2_cas.run()

converged SCF energy = -1.116998996754
CASCI E = -1.1373060356956  E(CI) = -1.85727503014458


In [ ]:
import qiskit.tools.jupyter

%qiskit_version_table